In [1]:
! pip install torch
! pip install transformers
! pip install scikit-learn
! pip install tqdm
! pip install numpy
! pip install datasets
! pip install nltk
import nltk
nltk.download('stopwords')
! pip install scipy
! pip install transformers[torch] accelerate

! pip install datasets
import json
from datasets import Dataset

import pandas as pd
! pip install nlpaug

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 6.3 MB/s eta 0:00:00


In [2]:
##Unfair-TOS
# Load JSON data from file
file_path = '/content/sample_data/incorrect_predictions_dataset (4).json'
with open(file_path, 'r') as json_file:
    data = json.load(json_file)

# Create a dictionary to store original_labels
labels_dict = {}
text_dict ={}
labels =[]

# Iterate through the data and extract original_labels
for entry in data:
    if "original_labels" in entry:
        original_labels = entry["original_labels"]
        if "Index" in entry:
            index = entry["Index"]
            labels_dict[index] = original_labels
        if "text" in entry:
            text = entry["text"]
            text_dict[index]={
                "original_labels": original_labels,
                "text": text
            }
        elif len(labels_dict) > 0:
            last_index = max(labels_dict.keys())
            labels_dict[last_index].extend(original_labels)
        labels.append(original_labels)

print("*********")
print(len(text_dict))
# Initialize lists to store texts and labels
texts = []
text= []
#labels = label_lists[0:173]
print("#####")
print(len(labels))
# Iterate through the data and extract relevant information
for entry in data:
    if "text" in entry:
        text = entry["text"]
        if "Index" in entry:
            index = entry["Index"]
            #labels.append(labels_dict.get(index, []))
        else:
            # Use the last index available in labels_dict
            last_index = max(labels_dict.keys())
            #labels.append(labels_dict.get(last_index, []))
        texts.append(text)

print("Number of Texts:", len(texts))
print("Number of Labels:", len(labels))

original_label_counts = {}
for label_list in labels:
    for label in label_list:
        if label in original_label_counts:
            original_label_counts[label] += 1
        else:
            original_label_counts[label] = 1

# Find the most occurring original label
most_common_original_label = max(original_label_counts, key=original_label_counts.get)
print("Most Common Original Label:", most_common_original_label)
print("Occurrences:", original_label_counts[most_common_original_label])


first_occurrence_index = None
for index, label_list in enumerate(labels):
    if most_common_original_label in label_list:
        first_occurrence_index = index
        break

print("First Occurrence Index of Label 2:", first_occurrence_index)

print("Text:", texts[first_occurrence_index])


*********
0
#####
171
Number of Texts: 171
Number of Labels: 171
Most Common Original Label: 2
Occurrences: 38
First Occurrence Index of Label 2: 2
Text: academia.edu reserves the right , at its sole discretion , to modify the site , services and these terms , at any time and without prior notice .


In [8]:
import nlpaug.augmenter.word as naw
import random

# Most common original label and corresponding text
most_common_original_label = most_common_original_label
corresponding_text = texts[first_occurrence_index]

# Initialize augmenter
aug = naw.SynonymAug(aug_src='wordnet', aug_p=0.3)

# Apply data augmentation to the corresponding text
augmented_texts = [aug.augment(corresponding_text) for _ in range(50)]  # Generate 5 augmented examples

# Print augmented texts
print("Original Text:", corresponding_text)
print(len(augmented_texts))

augmented_labels=[]
for i in range(len(augmented_texts)+1):
    augmented_labels.append([most_common_original_label])

print(len(augmented_labels))

Original Text: academia.edu reserves the right , at its sole discretion , to modify the site , services and these terms , at any time and without prior notice .
50
51


In [9]:
from datasets import load_dataset
existing_dataset = load_dataset("lex_glue", 'unfair_tos')
augmented_texts_flat = [text[0] for text in augmented_texts]
data = {
    "text": [corresponding_text] + augmented_texts_flat,
    "label": [most_common_original_label] * (len(augmented_texts) + 1)  # Use the most common label
}
augmented_dataset = Dataset.from_dict(data)

# Create a Dataset instance
augmented_dataset = Dataset.from_dict(data)
merged_dataset = Dataset.from_dict({
    "text": existing_dataset["train"]["text"] + augmented_dataset["text"],
    "labels": existing_dataset["train"]["labels"] + augmented_labels,
})

In [5]:
from dataclasses import dataclass
from typing import Optional, Tuple

import torch
import numpy as np
from torch import nn
from transformers.file_utils import ModelOutput


@dataclass
class SimpleOutput(ModelOutput):
    last_hidden_state: torch.FloatTensor = None
    past_key_values: Optional[Tuple[Tuple[torch.FloatTensor]]] = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None
    cross_attentions: Optional[Tuple[torch.FloatTensor]] = None


def sinusoidal_init(num_embeddings: int, embedding_dim: int):
    # keep dim 0 for padding token position encoding zero vector
    position_enc = np.array([
        [pos / np.power(10000, 2 * i / embedding_dim) for i in range(embedding_dim)]
        if pos != 0 else np.zeros(embedding_dim) for pos in range(num_embeddings)])

    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2])  # dim 2i
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2])  # dim 2i+1
    return torch.from_numpy(position_enc).type(torch.FloatTensor)


class HierarchicalBert(nn.Module):

    def __init__(self, encoder, max_segments=64, max_segment_length=128):
        super(HierarchicalBert, self).__init__()
        supported_models = ['bert', 'roberta', 'deberta']
        assert encoder.config.model_type in supported_models  # other model types are not supported so far
        # Pre-trained segment (token-wise) encoder, e.g., BERT
        self.encoder = encoder
        # Specs for the segment-wise encoder
        self.hidden_size = encoder.config.hidden_size
        self.max_segments = max_segments
        self.max_segment_length = max_segment_length
        # Init sinusoidal positional embeddings
        self.seg_pos_embeddings = nn.Embedding(max_segments + 1, encoder.config.hidden_size,
                                               padding_idx=0,
                                               _weight=sinusoidal_init(max_segments + 1, encoder.config.hidden_size))
        # Init segment-wise transformer-based encoder
        self.seg_encoder = nn.Transformer(d_model=encoder.config.hidden_size,
                                          nhead=encoder.config.num_attention_heads,
                                          batch_first=True, dim_feedforward=encoder.config.intermediate_size,
                                          activation=encoder.config.hidden_act,
                                          dropout=encoder.config.hidden_dropout_prob,
                                          layer_norm_eps=encoder.config.layer_norm_eps,
                                          num_encoder_layers=2, num_decoder_layers=0).encoder

    def forward(self,
                input_ids=None,
                attention_mask=None,
                token_type_ids=None,
                position_ids=None,
                head_mask=None,
                inputs_embeds=None,
                labels=None,
                output_attentions=None,
                output_hidden_states=None,
                return_dict=None,
                ):
        # Hypothetical Example
        # Batch of 4 documents: (batch_size, n_segments, max_segment_length) --> (4, 64, 128)
        # BERT-BASE encoder: 768 hidden units

        # Squash samples and segments into a single axis (batch_size * n_segments, max_segment_length) --> (256, 128)
        input_ids_reshape = input_ids.contiguous().view(-1, input_ids.size(-1))
        attention_mask_reshape = attention_mask.contiguous().view(-1, attention_mask.size(-1))
        if token_type_ids is not None:
            token_type_ids_reshape = token_type_ids.contiguous().view(-1, token_type_ids.size(-1))
        else:
            token_type_ids_reshape = None

        # Encode segments with BERT --> (256, 128, 768)
        encoder_outputs = self.encoder(input_ids=input_ids_reshape,
                                       attention_mask=attention_mask_reshape,
                                       token_type_ids=token_type_ids_reshape)[0]

        # Reshape back to (batch_size, n_segments, max_segment_length, output_size) --> (4, 64, 128, 768)
        encoder_outputs = encoder_outputs.contiguous().view(input_ids.size(0), self.max_segments,
                                                            self.max_segment_length,
                                                            self.hidden_size)

        # Gather CLS outputs per segment --> (4, 64, 768)
        encoder_outputs = encoder_outputs[:, :, 0]

        # Infer real segments, i.e., mask paddings
        seg_mask = (torch.sum(input_ids, 2) != 0).to(input_ids.dtype)
        # Infer and collect segment positional embeddings
        seg_positions = torch.arange(1, self.max_segments + 1).to(input_ids.device) * seg_mask
        # Add segment positional embeddings to segment inputs
        encoder_outputs += self.seg_pos_embeddings(seg_positions)

        # Encode segments with segment-wise transformer
        seg_encoder_outputs = self.seg_encoder(encoder_outputs)

        # Collect document representation
        outputs, _ = torch.max(seg_encoder_outputs, 1)

        return SimpleOutput(last_hidden_state=outputs, hidden_states=outputs)


if __name__ == "__main__":
    from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
    tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

    # Use as a stand-alone encoder
    bert = AutoModel.from_pretrained('bert-base-uncased')
    model = HierarchicalBert(encoder=bert, max_segments=64, max_segment_length=128)

    fake_inputs = {'input_ids': [], 'attention_mask': [], 'token_type_ids': []}
    for i in range(4):
        # Tokenize segment
        temp_inputs = tokenizer(['dog ' * 126] * 64)
        fake_inputs['input_ids'].append(temp_inputs['input_ids'])
        fake_inputs['attention_mask'].append(temp_inputs['attention_mask'])
        fake_inputs['token_type_ids'].append(temp_inputs['token_type_ids'])

    fake_inputs['input_ids'] = torch.as_tensor(fake_inputs['input_ids'])
    fake_inputs['attention_mask'] = torch.as_tensor(fake_inputs['attention_mask'])
    fake_inputs['token_type_ids'] = torch.as_tensor(fake_inputs['token_type_ids'])

    output = model(fake_inputs['input_ids'], fake_inputs['attention_mask'], fake_inputs['token_type_ids'])

    # 4 document representations of 768 features are expected
    assert output[0].shape == torch.Size([4, 768])

    # Use with HuggingFace AutoModelForSequenceClassification and Trainer API

    # Init Classifier
    model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=10)
    # Replace flat BERT encoder with hierarchical BERT encoder
    model.bert = HierarchicalBert(encoder=model.bert, max_segments=64, max_segment_length=128)
    output = model(fake_inputs['input_ids'], fake_inputs['attention_mask'], fake_inputs['token_type_ids'])

    # 4 document outputs with 10 (num_labels) logits are expected
    assert output.logits.shape == torch.Size([4, 10])



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from torch import nn
from transformers import Trainer


class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits.view(-1, self.model.config.num_labels),
                        labels.float().view(-1, self.model.config.num_labels))
        return (loss, outputs) if return_outputs else loss





In [10]:
#!/usr/bin/env python
# coding=utf-8
""" Finetuning models on UNFAIR-ToC (e.g. Bert, RoBERTa, LEGAL-BERT)."""

import logging
import os
import random
import sys
from dataclasses import dataclass, field
from typing import Optional

import datasets
from datasets import load_dataset
from sklearn.metrics import f1_score

from scipy.special import expit
import glob
import shutil
import numpy as np

import transformers
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    TrainingArguments,
    default_data_collator,
    set_seed,
    EarlyStoppingCallback,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from transformers.utils.versions import require_version


# Will error if the minimal version of Transformers is not installed. Remove at your own risks.
check_min_version("4.9.0")

require_version("datasets>=1.8.0", "To fix: pip install -r examples/pytorch/text-classification/requirements.txt")

logger = logging.getLogger(__name__)


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.

    Using `HfArgumentParser` we can turn this class
    into argparse arguments to be able to specify them on
    the command line.
    """

    max_seq_length: Optional[int] = field(
        default=128,
        metadata={
            "help": "The maximum total input sequence length after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached preprocessed datasets or not."}
    )
    pad_to_max_length: bool = field(
        default=True,
        metadata={
            "help": "Whether to pad all samples to `max_seq_length`. "
            "If False, will pad the samples dynamically when batching to the maximum length in the batch."
        },
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of training examples to this "
            "value if set."
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
            "value if set."
        },
    )
    max_predict_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of prediction examples to this "
            "value if set."
        },
    )
    server_ip: Optional[str] = field(default=None, metadata={"help": "For distant debugging."})
    server_port: Optional[str] = field(default=None, metadata={"help": "For distant debugging."})


@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        default=None, metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where do you want to store the pretrained models downloaded from huggingface.co"},
    )
    do_lower_case: Optional[bool] = field(
        default=True,
        metadata={"help": "arg to indicate if tokenizer should do lower case in AutoTokenizer.from_pretrained()"},
    )
    use_fast_tokenizer: bool = field(
        default=True,
        metadata={"help": "Whether to use one of the fast tokenizer (backed by the tokenizers library) or not."},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "The specific model version to use (can be a branch name, tag name or commit id)."},
    )
    use_auth_token: bool = field(
        default=False,
        metadata={
            "help": "Will use the token generated when running `transformers-cli login` (necessary to use this script "
            "with private models)."
        },
    )


def main(training_args):
    # See all possible arguments in src/transformers/training_args.py
    # or by passing the --help flag to this script.
    # We now keep distinct sets of args, for a cleaner separation of concerns.

    model_args = ModelArguments(
        model_name_or_path="nlpaueb/legal-bert-base-uncased",
        #hierarchical=True,
        do_lower_case=True,
        use_fast_tokenizer=True,
    )
    data_args = DataTrainingArguments(
        max_seq_length=128,
        #max_segments=64,
        #max_seg_length=128,
        overwrite_cache=False,
        pad_to_max_length=True,
    )

    # Setup distant debugging if needed
    if data_args.server_ip and data_args.server_port:
        # Distant debugging - see https://code.visualstudio.com/docs/python/debugging#_attach-to-a-local-script
        import ptvsd

        print("Waiting for debugger attach")
        ptvsd.enable_attach(address=(data_args.server_ip, data_args.server_port), redirect_output=True)
        ptvsd.wait_for_attach()

    # Fix boolean parameter
    if model_args.do_lower_case == 'False' or not model_args.do_lower_case:
        model_args.do_lower_case = False
        'Tokenizer do_lower_case False'
    else:
        model_args.do_lower_case = True

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        handlers=[logging.StreamHandler(sys.stdout)],
    )

    log_level = training_args.get_process_log_level()
    logger.setLevel(log_level)
    datasets.utils.logging.set_verbosity(log_level)
    transformers.utils.logging.set_verbosity(log_level)
    transformers.utils.logging.enable_default_handler()
    transformers.utils.logging.enable_explicit_format()

    # Log on each process the small summary:
    logger.warning(
        f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
        + f"distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
    )
    logger.info(f"Training/evaluation parameters {training_args}")

    # Detecting last checkpoint.
    last_checkpoint = None
    if os.path.isdir(training_args.output_dir) and training_args.do_train and not training_args.overwrite_output_dir:
        last_checkpoint = get_last_checkpoint(training_args.output_dir)
        if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
            raise ValueError(
                f"Output directory ({training_args.output_dir}) already exists and is not empty. "
                "Use --overwrite_output_dir to overcome."
            )
        elif last_checkpoint is not None:
            logger.info(
                f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
                "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
            )

    # Set seed before initializing model.
    set_seed(training_args.seed)

    # In distributed training, the load_dataset function guarantees that only one local process can concurrently
    # download the dataset.
    # Downloading and loading eurlex dataset from the hub.
    if training_args.do_train:
        train_dataset = merged_dataset

    if training_args.do_eval:
        eval_dataset = load_dataset("lex_glue", "unfair_tos", split="validation", data_dir='data', cache_dir=model_args.cache_dir)

    if training_args.do_predict:
        predict_dataset = load_dataset("lex_glue", "unfair_tos", split="test", data_dir='data', cache_dir=model_args.cache_dir)

    # Labels
    label_list = list(range(8))
    num_labels = len(label_list)

    # Load pretrained model and tokenizer
    # In distributed training, the .from_pretrained methods guarantee that only one local process can concurrently
    # download model & vocab.
    config = AutoConfig.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        num_labels=num_labels,
        finetuning_task="unfair_toc",
        cache_dir=model_args.cache_dir,
        revision=model_args.model_revision,
        use_auth_token=True if model_args.use_auth_token else None,
    )

    if config.model_type == 'big_bird':
        config.attention_type = 'original_full'

    if config.model_type == 'longformer':
        config.attention_window = [128] * config.num_hidden_layers

    tokenizer = AutoTokenizer.from_pretrained(
        model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
        do_lower_case=model_args.do_lower_case,
        cache_dir=model_args.cache_dir,
        use_fast=model_args.use_fast_tokenizer,
        revision=model_args.model_revision,
        use_auth_token=True if model_args.use_auth_token else None,
    )
    model = AutoModelForSequenceClassification.from_pretrained(
        model_args.model_name_or_path,
        from_tf=bool(".ckpt" in model_args.model_name_or_path),
        config=config,
        cache_dir=model_args.cache_dir,
        revision=model_args.model_revision,
        use_auth_token=True if model_args.use_auth_token else None,
    )

    # Preprocessing the datasets
    # Padding strategy
    if data_args.pad_to_max_length:
        padding = "max_length"
    else:
        # We will pad later, dynamically at batch creation, to the max sequence length in each batch
        padding = False

    def preprocess_function(examples):
        # Tokenize the texts
        batch = tokenizer(
            examples["text"],
            padding=padding,
            max_length=data_args.max_seq_length,
            truncation=True,
        )
        batch["labels"] = [[1 if label in labels else 0 for label in label_list] for labels in
                              examples["labels"]]

        return batch

    if training_args.do_train:

        if data_args.max_train_samples is not None:
            train_dataset = train_dataset.select(range(data_args.max_train_samples))
            print("#################")
            print (train_dataset('labels'))

        with training_args.main_process_first(desc="train dataset map pre-processing"):
            train_dataset = train_dataset.map(
                preprocess_function,
                batched=True,
                load_from_cache_file=not data_args.overwrite_cache,
                desc="Running tokenizer on train dataset",
            )
        # Log a few random samples from the training set:
        for index in random.sample(range(len(train_dataset)), 3):
            logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

    if training_args.do_eval:
        if data_args.max_eval_samples is not None:
            eval_dataset = eval_dataset.select(range(data_args.max_eval_samples))
        with training_args.main_process_first(desc="validation dataset map pre-processing"):
            eval_dataset = eval_dataset.map(
                preprocess_function,
                batched=True,
                load_from_cache_file=not data_args.overwrite_cache,
                desc="Running tokenizer on validation dataset",
            )

    if training_args.do_predict:
        if data_args.max_predict_samples is not None:
            predict_dataset = predict_dataset.select(range(data_args.max_predict_samples))
        with training_args.main_process_first(desc="prediction dataset map pre-processing"):
            predict_dataset = predict_dataset.map(
                preprocess_function,
                batched=True,
                load_from_cache_file=not data_args.overwrite_cache,
                desc="Running tokenizer on prediction dataset",
            )

    # You can define your custom compute_metrics function. It takes an `EvalPrediction` object (a namedtuple with a
    # predictions and label_ids field) and has to return a dictionary string to float.
    def compute_metrics(p: EvalPrediction):
        # Fix gold labels
        y_true = np.zeros((p.label_ids.shape[0], p.label_ids.shape[1] + 1), dtype=np.int32)
        y_true[:, :-1] = p.label_ids
        y_true[:, -1] = (np.sum(p.label_ids, axis=1) == 0).astype('int32')
        # Fix predictions
        logits = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
        preds = (expit(logits) > 0.5).astype('int32')
        y_pred = np.zeros((p.label_ids.shape[0], p.label_ids.shape[1] + 1), dtype=np.int32)
        y_pred[:, :-1] = preds
        y_pred[:, -1] = (np.sum(preds, axis=1) == 0).astype('int32')
        # Compute scores
        macro_f1 = f1_score(y_true=y_true, y_pred=y_pred, average='macro', zero_division=0)
        micro_f1 = f1_score(y_true=y_true, y_pred=y_pred, average='micro', zero_division=0)
        return {'macro-f1': macro_f1, 'micro-f1': micro_f1}

    # Data collator will default to DataCollatorWithPadding, so we change it if we already did the padding.
    if data_args.pad_to_max_length:
        data_collator = default_data_collator
    elif training_args.fp16:
        data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8)
    else:
        data_collator = None

    # Initialize our Trainer
    trainer = MultilabelTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset if training_args.do_train else None,
        eval_dataset=eval_dataset if training_args.do_eval else None,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
        data_collator=data_collator,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=15)]
    )

    # Training
    if training_args.do_train:
        checkpoint = None
        if training_args.resume_from_checkpoint is not None:
            checkpoint = training_args.resume_from_checkpoint
        elif last_checkpoint is not None:
            checkpoint = last_checkpoint
        train_result = trainer.train(resume_from_checkpoint=checkpoint)
        metrics = train_result.metrics
        max_train_samples = (
            data_args.max_train_samples if data_args.max_train_samples is not None else len(train_dataset)
        )
        metrics["train_samples"] = min(max_train_samples, len(train_dataset))

        trainer.save_model()  # Saves the tokenizer too for easy upload

        trainer.log_metrics("train", metrics)
        trainer.save_metrics("train", metrics)
        trainer.save_state()

    # Evaluation
    if training_args.do_eval:
        logger.info("*** Evaluate ***")
        metrics = trainer.evaluate(eval_dataset=eval_dataset)

        max_eval_samples = data_args.max_eval_samples if data_args.max_eval_samples is not None else len(eval_dataset)
        metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))

        trainer.log_metrics("eval", metrics)
        trainer.save_metrics("eval", metrics)

    # Prediction
    if training_args.do_predict:
        logger.info("*** Predict ***")
        predictions, labels, metrics = trainer.predict(predict_dataset, metric_key_prefix="predict")

        max_predict_samples = (
            data_args.max_predict_samples if data_args.max_predict_samples is not None else len(predict_dataset)
        )
        metrics["predict_samples"] = min(max_predict_samples, len(predict_dataset))

        trainer.log_metrics("predict", metrics)
        trainer.save_metrics("predict", metrics)

        output_predict_file = os.path.join(training_args.output_dir, "test_predictions.csv")
        if trainer.is_world_process_zero():
            with open(output_predict_file, "w") as writer:
                if isinstance(predictions[0], np.float32):
                   pred_line = f'{predictions[0]:.5f}'
                   writer.write(pred_line)
                else:
                    for key, value in metrics.items():
                        if isinstance(value, float):
                           writer.write(f"{key} = {value:.5f}\n")
                        else:
                           writer.write(f"{key} = {value}\n")


    # Clean up checkpoints
    checkpoints = [filepath for filepath in glob.glob(f'{training_args.output_dir}/*/') if '/checkpoint' in filepath]
    for checkpoint in checkpoints:
        shutil.rmtree(checkpoint)


if __name__ == "__main__":
 #    For Evaluation
    training_args = TrainingArguments(
        do_train = True,
        do_eval = True,
        do_predict = True,
        output_dir=os.getcwd(),
        overwrite_output_dir=True,
        num_train_epochs=20,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        save_steps=500,
        save_total_limit=2,
        fp16=False,
        logging_dir="./logs",
        logging_steps=200,
        evaluation_strategy="steps",
        eval_steps=500,
        logging_first_step=False,
        load_best_model_at_end = True,
        metric_for_best_model="micro-f1",
    )
    main(training_args)


[WARNING|modeling_utils.py:3553] 2023-09-03 17:57:56,109 >> Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running tokenizer on train dataset:   0%|          | 0/5583 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss,Macro-f1,Micro-f1
500,0.065600,0.039637,0.354686,0.922808
1000,0.032700,0.030331,0.582154,0.926136
1500,0.027200,0.026419,0.725193,0.943644
2000,0.021100,0.025829,0.733401,0.942152
2500,0.019700,0.027778,0.750638,0.946568
3000,0.010300,0.026271,0.767480,0.951363
3500,0.009700,0.028838,0.742079,0.945352
4000,0.005300,0.032501,0.738618,0.945852
4500,0.005100,0.037548,0.747403,0.949367
5000,0.003400,0.032396,0.778011,0.948148


***** train metrics *****
  epoch                    =       20.0
  total_flos               =  6840695GF
  train_loss               =     0.0091
  train_runtime            = 0:21:24.69
  train_samples            =       5583
  train_samples_per_second =     86.916
  train_steps_per_second   =     10.866


***** eval metrics *****
  epoch                   =       20.0
  eval_loss               =     0.0399
  eval_macro-f1           =     0.7732
  eval_micro-f1           =     0.9542
  eval_runtime            = 0:00:06.13
  eval_samples            =       2275
  eval_samples_per_second =    371.002
  eval_steps_per_second   =     46.477
***** predict metrics *****
  predict_loss               =     0.0384
  predict_macro-f1           =     0.8296
  predict_micro-f1           =      0.958
  predict_runtime            = 0:00:04.52
  predict_samples            =       1607
  predict_samples_per_second =    355.192
  predict_steps_per_second   =     44.427


In [ ]:
##LEDGAR
##Unfair-TOS
# Load JSON data from file
file_path = '/content/sample_data/incorrect_predictions_dataset (5).json'

with open(file_path, 'r') as json_file:
    data = json.load(json_file)

# Create a dictionary to store original_labels
labels_dict = {}
text_dict ={}
labels =[]

# Iterate through the data and extract original_labels
for entry in data:
    if "original_labels" in entry:
        original_labels = entry["original_labels"]
        if "Index" in entry:
            index = entry["Index"]
            labels_dict[index] = original_labels
        if "text" in entry:
            text = entry["text"]
            text_dict[index]={
                "original_labels": original_labels,
                "text": text
            }
        elif len(labels_dict) > 0:
            last_index = max(labels_dict.keys())
            labels_dict[last_index].extend(original_labels)
        labels.append(original_labels)

print("*********")
print(len(text_dict))
# Initialize lists to store texts and labels
texts = []
text= []
#labels = label_lists[0:173]
print("#####")
print(len(labels))
# Iterate through the data and extract relevant information
for entry in data:
    if "text" in entry:
        text = entry["text"]
        if "Index" in entry:
            index = entry["Index"]
            #labels.append(labels_dict.get(index, []))
        else:
            # Use the last index available in labels_dict
            last_index = max(labels_dict.keys())
            #labels.append(labels_dict.get(last_index, []))
        texts.append(text)

print("Number of Texts:", len(texts))
print("Number of Labels:", len(labels))

original_label_counts = {}
for label_list in labels:
    for label in label_list:
        if label in original_label_counts:
            original_label_counts[label] += 1
        else:
            original_label_counts[label] = 1

# Find the most occurring original label
most_common_original_label = max(original_label_counts, key=original_label_counts.get)
print("Most Common Original Label:", most_common_original_label)
print("Occurrences:", original_label_counts[most_common_original_label])


first_occurrence_index = None
for index, label_list in enumerate(labels):
    if most_common_original_label in label_list:
        first_occurrence_index = index
        break

print("First Occurrence Index of Label 2:", first_occurrence_index)

print("Text:", texts[first_occurrence_index])


*********
0
#####
10000
Number of Texts: 10000
Number of Labels: 10000
Most Common Original Label: 47
Occurrences: 582
First Occurrence Index of Label 2: 14
Text: The validity and interpretation of this Agreement, and the terms and conditions set forth herein, shall be construed and interpreted in accordance with and governed by the laws of the State of California, without giving effect to any provisions relating to conflict of laws.


In [ ]:
##EcTHR(B)
# Load JSON data from file
file_path = '/content/sample_data/incorrect_predictions_dataset (3).json'

with open(file_path, 'r') as json_file:
    data = json.load(json_file)

# Create a dictionary to store original_labels
labels_dict = {}
text_dict ={}
labels =[]

# Iterate through the data and extract original_labels
for entry in data:
    if "original_labels" in entry:
        original_labels = entry["original_labels"]
        if "Index" in entry:
            index = entry["Index"]
            labels_dict[index] = original_labels
        if "text" in entry:
            text = entry["text"]
            text_dict[index]={
                "original_labels": original_labels,
                "text": text
            }
        elif len(labels_dict) > 0:
            last_index = max(labels_dict.keys())
            labels_dict[last_index].extend(original_labels)
        labels.append(original_labels)

print("*********")
print(len(text_dict))
# Initialize lists to store texts and labels
texts = []
text= []
#labels = label_lists[0:173]
print("#####")
print(len(labels))
# Iterate through the data and extract relevant information
for entry in data:
    if "text" in entry:
        text = entry["text"]
        if "Index" in entry:
            index = entry["Index"]
            #labels.append(labels_dict.get(index, []))
        else:
            # Use the last index available in labels_dict
            last_index = max(labels_dict.keys())
            #labels.append(labels_dict.get(last_index, []))
        texts.append(text)

print("Number of Texts:", len(texts))
print("Number of Labels:", len(labels))

original_label_counts = {}
for label_list in labels:
    for label in label_list:
        if label in original_label_counts:
            original_label_counts[label] += 1
        else:
            original_label_counts[label] = 1

# Find the most occurring original label
most_common_original_label = max(original_label_counts, key=original_label_counts.get)
print("Most Common Original Label:", most_common_original_label)
print("Occurrences:", original_label_counts[most_common_original_label])


first_occurrence_index = None
for index, label_list in enumerate(labels):
    if most_common_original_label in label_list:
        first_occurrence_index = index
        break

print("First Occurrence Index of Label :", first_occurrence_index)

print("Text:", texts[first_occurrence_index])


*********
0
#####
1000
Number of Texts: 1000
Number of Labels: 1000
Most Common Original Label: 3
Occurrences: 394
First Occurrence Index of Label : 2
Text: ['5.  The applicant was born in 1965 and lives in Smědčice.', '6.  On 9 November 2006 the applicant requested a building permit for temporary stables for horses. On 6 January 2011 the Rokycany Planning Office (stavební úřad) dismissed his request and on 26 May 2011 the Plzeň Regional Office (krajský úřad) upheld that decision.', '7.  On 29 March 2013 the Plzeň Regional Court (krajský soud) dismissed a complaint lodged by the applicant against the decision of the Plzeň Regional Office.', '8.  On 31 July 2013 the Supreme Administrative Court (Nejvyšší správní soud) dismissed an appeal on points of law lodged by the applicant. The decision was served on the applicant on 28 August 2013.', '9.  On 29 October 2013 the applicant lodged a constitutional complaint (ústavní stížnost).', '10.  On 31 March 2014 the Constitutional Court (Ústavn

In [ ]:
##EctHR(A)
# Load JSON data from file
file_path = '/content/sample_data/incorrect_predictions_dataset (6).json'

with open(file_path, 'r') as json_file:
    data = json.load(json_file)

# Create a dictionary to store original_labels
labels_dict = {}
text_dict ={}
labels =[]

# Iterate through the data and extract original_labels
for entry in data:
    if "original_labels" in entry:
        original_labels = entry["original_labels"]
        if "Index" in entry:
            index = entry["Index"]
            labels_dict[index] = original_labels
        if "text" in entry:
            text = entry["text"]
            text_dict[index]={
                "original_labels": original_labels,
                "text": text
            }
        elif len(labels_dict) > 0:
            last_index = max(labels_dict.keys())
            labels_dict[last_index].extend(original_labels)
        labels.append(original_labels)

print("*********")
print(len(text_dict))
# Initialize lists to store texts and labels
texts = []
text= []
#labels = label_lists[0:173]
print("#####")
print(len(labels))
# Iterate through the data and extract relevant information
for entry in data:
    if "text" in entry:
        text = entry["text"]
        if "Index" in entry:
            index = entry["Index"]
            #labels.append(labels_dict.get(index, []))
        else:
            # Use the last index available in labels_dict
            last_index = max(labels_dict.keys())
            #labels.append(labels_dict.get(last_index, []))
        texts.append(text)

print("Number of Texts:", len(texts))
print("Number of Labels:", len(labels))

original_label_counts = {}
for label_list in labels:
    for label in label_list:
        if label in original_label_counts:
            original_label_counts[label] += 1
        else:
            original_label_counts[label] = 1

# Find the most occurring original label
most_common_original_label = max(original_label_counts, key=original_label_counts.get)
print("Most Common Original Label:", most_common_original_label)
print("Occurrences:", original_label_counts[most_common_original_label])


first_occurrence_index = None
for index, label_list in enumerate(labels):
    if most_common_original_label in label_list:
        first_occurrence_index = index
        break

print("First Occurrence Index of Label :", first_occurrence_index)

print("Text:", texts[first_occurrence_index])


*********
0
#####
1000
Number of Texts: 1000
Number of Labels: 1000
Most Common Original Label: 3
Occurrences: 299
First Occurrence Index of Label : 2
Text: ['5.  The applicant was born in 1965 and lives in Smědčice.', '6.  On 9 November 2006 the applicant requested a building permit for temporary stables for horses. On 6 January 2011 the Rokycany Planning Office (stavební úřad) dismissed his request and on 26 May 2011 the Plzeň Regional Office (krajský úřad) upheld that decision.', '7.  On 29 March 2013 the Plzeň Regional Court (krajský soud) dismissed a complaint lodged by the applicant against the decision of the Plzeň Regional Office.', '8.  On 31 July 2013 the Supreme Administrative Court (Nejvyšší správní soud) dismissed an appeal on points of law lodged by the applicant. The decision was served on the applicant on 28 August 2013.', '9.  On 29 October 2013 the applicant lodged a constitutional complaint (ústavní stížnost).', '10.  On 31 March 2014 the Constitutional Court (Ústavn